# TODO

# Set-up

In [1]:
import os
import sys
import glob
import time
import yaml
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import sparse
import scanpy as sc
import loompy as lp
sc.settings.verbosity = 3

sys.path.append("/cellar/users/aklie/data/igvf/bin")
from utils import make_dirs

sys.path.append("/cellar/users/aklie/data/igvf/bin/data_wrangling")
from write import write_10x

/cellar/users/aklie/opt/miniconda3/envs/scverse-lite-py38/lib/python3.8/site-packages/loompy/bus_file.py:68: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def twobit_to_dna(twobit: int, size: int) -> str:
/cellar/users/aklie/opt/miniconda3/envs/scverse-lite-py38/lib/python3.8/site-packages/loompy/bus_file.py:85: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def 

# Load file metadata

In [2]:
# Make the notebook directory the cwd
os.chdir("/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/wgcna/bin/Bridge_Satpathy")
os.getcwd()

'/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/wgcna/bin/Bridge_Satpathy'

In [3]:
# Where the input metadata is stored for all files to run on
in_dir = "/cellar/users/aklie/data/igvf/topic_grn_links/subsets/for_regulon_inference/Bridge_Satpathy/29Aug23/3744376661_20230829-175127"

In [4]:
# Load the file metadata
file_metadata = pd.read_csv(os.path.join(in_dir, "file_metadata.tsv"), sep="\t")
file_metadata.head()

,file,file_type,cells,genes,normalization,n_cells,n_genes,percent_int,percent_0,percent_less_than_0
0,random_500.random_2000.raw.h5ad,.h5ad,random_500,random_2000,raw,500,2000,0.995209,0.951713,0.0
1,random_500.random_2000.raw.loom,.loom,random_500,random_2000,raw,500,2000,0.995209,0.951713,0.0
2,random_500.random_2000.raw.tsv,.tsv,random_500,random_2000,raw,2000,500,0.995209,0.951713,0.0
3,random_500.random_2000.log1p_cp10k.h5ad,.h5ad,random_500,random_2000,log1p_cp10k,500,2000,0.951713,0.951713,0.0
4,random_500.random_2000.log1p_cp10k.loom,.loom,random_500,random_2000,log1p_cp10k,500,2000,0.951713,0.951713,0.0


# Extract runs of interest

In [5]:
# Parameters for runs of interest
genes = ["0.05"]  # could be ["0.01", "0.05", "0.1", "0.2"]
cells = ["balanced_genotype_microglia"]
normalizations = ["raw", "cpm_rank"]  # could be ["raw", "log1p_cp10k", "pf_log1p_pf", "cpm_rank"]
file_types = [".h5ad"]  # could be [".h5ad", ".loom", ".tsv"], but selecting tsv for GENIE3
out_dir = "/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/wgcna/results/Bridge_Satpathy"

In [6]:
# Set up masks for the file metadata based on above
genes_mask = file_metadata["genes"].isin(genes)
cells_mask = file_metadata["cells"].isin(cells)
normalizations_mask = file_metadata["normalization"].isin(normalizations)
file_types_mask = file_metadata["file_type"].isin(file_types)

In [7]:
# Extract the files to run
curr_metadata = file_metadata[genes_mask & cells_mask & normalizations_mask & file_types_mask]
curr_metadata["in_file"] = curr_metadata["cells"] + "/" + curr_metadata["genes"].astype(str) + "/" + curr_metadata["normalization"] + "/" + curr_metadata["file"]
curr_metadata["in_file_path"] = curr_metadata["in_file"].apply(lambda x: os.path.join(in_dir, x))
curr_metadata["genotype"] = [i.split(".")[0].split('-')[-1] for i in curr_metadata["file"]] 
curr_metadata["out_dir"] = out_dir
curr_metadata["out_dir_path"] = curr_metadata["out_dir"] + "/" + curr_metadata["cells"] + "/" + curr_metadata["genes"].astype(str) + "/" + curr_metadata["normalization"] + "/" + curr_metadata["genotype"]
curr_metadata

/tmp/ipykernel_905914/441207337.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_metadata["in_file"] = curr_metadata["cells"] + "/" + curr_metadata["genes"].astype(str) + "/" + curr_metadata["normalization"] + "/" + curr_metadata["file"]
/tmp/ipykernel_905914/441207337.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_metadata["in_file_path"] = curr_metadata["in_file"].apply(lambda x: os.path.join(in_dir, x))
/tmp/ipykernel_905914/441207337.py:5: SettingWithCopyWarning: 
A value is trying to 

,file,file_type,cells,genes,normalization,n_cells,n_genes,percent_int,percent_0,percent_less_than_0,in_file,in_file_path,genotype,out_dir,out_dir_path
480,balanced_genotype_microglia-B6J.0.05.raw.h5ad,.h5ad,balanced_genotype_microglia,0.05,raw,1498,6277,0.987272,0.873955,0.000000,balanced_genotype_microglia/0.05/raw/balanced_...,/cellar/users/aklie/data/igvf/topic_grn_links/...,B6J,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/projects/igvf/topic_grn_li...
481,balanced_genotype_microglia-CASTJ.0.05.raw.h5ad,.h5ad,balanced_genotype_microglia,0.05,raw,1498,6277,0.982353,0.853771,0.000000,balanced_genotype_microglia/0.05/raw/balanced_...,/cellar/users/aklie/data/igvf/topic_grn_links/...,CASTJ,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/projects/igvf/topic_grn_li...
482,balanced_genotype_microglia-both.0.05.raw.h5ad,.h5ad,balanced_genotype_microglia,0.05,raw,2996,6277,0.984812,0.863863,0.000000,balanced_genotype_microglia/0.05/raw/balanced_...,/cellar/users/aklie/data/igvf/topic_grn_links/...,both,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/projects/igvf/topic_grn_li...
507,balanced_genotype_microglia-B6J.0.05.cpm_rank....,.h5ad,balanced_genotype_microglia,0.05,cpm_rank,1498,6277,0.010634,0.002413,0.498810,balanced_genotype_microglia/0.05/cpm_rank/bala...,/cellar/users/aklie/data/igvf/topic_grn_links/...,B6J,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/projects/igvf/topic_grn_li...
508,balanced_genotype_microglia-CASTJ.0.05.cpm_ran...,.h5ad,balanced_genotype_microglia,0.05,cpm_rank,1498,6277,0.010560,0.002531,0.498743,balanced_genotype_microglia/0.05/cpm_rank/bala...,/cellar/users/aklie/data/igvf/topic_grn_links/...,CASTJ,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/projects/igvf/topic_grn_li...
509,balanced_genotype_microglia-both.0.05.cpm_rank...,.h5ad,balanced_genotype_microglia,0.05,cpm_rank,2996,6277,0.010597,0.002472,0.498777,balanced_genotype_microglia/0.05/cpm_rank/bala...,/cellar/users/aklie/data/igvf/topic_grn_links/...,both,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/projects/igvf/topic_grn_li...


In [8]:
time_id = time.strftime("%Y%m%d-%H%M%S")
run_id =  random.getrandbits(32)
run = str(run_id) + "_" + time_id

In [9]:
curr_run = curr_metadata[["in_file_path", "out_dir_path"]]
curr_run["tf_list"] = "/cellar/users/aklie/opt/igvf-ucsd/genie3_pipeline/data/tf_lists/allTFs_mm.txt"
curr_run["network_type"] = "signed"
curr_run["layer"] = "normalized_counts"
curr_run["run_id"] = run
curr_run.to_csv(f"{run}_paths.csv", index=False)

/tmp/ipykernel_905914/712451517.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_run["tf_list"] = "/cellar/users/aklie/opt/igvf-ucsd/genie3_pipeline/data/tf_lists/allTFs_mm.txt"
/tmp/ipykernel_905914/712451517.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_run["network_type"] = "signed"
/tmp/ipykernel_905914/712451517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

# DONE

---

# Scratch

## Create Seurat readable files for these (deprecated thanks to PyWGCNA)

In [31]:
from scipy import io
def write_10x(adata, dirname, layer=None, use_raw=False):
    """Write AnnData object to 10x CellRanger v3 output

    Parameters
    ----------
    adata : AnnData
        The annotated data matrix of shape `n_obs` x `n_vars`.
        Rows correspond to cells and columns to genes.
    filename : str
        Filename to save to
    layer : str
        Layer to save instead of `X`. If `None`, `X` is saved.

    Returns
    -------
    None
    """
    if use_raw:
        adata = adata.raw.to_adata()  #only if adata has RAW saved and thats what you want!!
    if layer is not None:
        X = adata.layers[layer].T
    else:
        X = adata.X.T
    with open(os.path.join(dirname, 'barcodes.tsv'), 'w') as f:
        for item in adata.obs_names:
            f.write(item + '\n')
    with open(os.path.join(dirname, 'features.tsv'), 'w') as f:
        for item in ['\t'.join([x, x,'Gene Expression']) for x in adata.var_names]:
            f.write(item + '\n')
    io.mmwrite(os.path.join(dirname, 'matrix.mtx'), X)
    adata.obs.to_csv(os.path.join(dirname, 'metadata.csv'))

In [33]:
for row in curr_metadata.iterrows():
    print(row[1]["in_file_path"])
    print(row[1]["out_dir_path"])
    ad = sc.read_h5ad(row[1]["in_file_path"])
    make_dirs(row[1]["out_dir_path"])
    write_10x(ad, row[1]["out_dir_path"], layer="normalized_counts")
    os.system("gzip -r " + row[1]["out_dir_path"])
    print("Done with " + row[1]["in_file_path"])

/cellar/users/aklie/data/igvf/topic_grn_links/subsets/for_regulon_inference/Bridge_Satpathy/29Aug23/3744376661_20230829-175127/balanced_genotype_microglia/0.05/raw/balanced_genotype_microglia-B6J.0.05.raw.h5ad
/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/hdwgcna/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J
Done with /cellar/users/aklie/data/igvf/topic_grn_links/subsets/for_regulon_inference/Bridge_Satpathy/29Aug23/3744376661_20230829-175127/balanced_genotype_microglia/0.05/raw/balanced_genotype_microglia-B6J.0.05.raw.h5ad
/cellar/users/aklie/data/igvf/topic_grn_links/subsets/for_regulon_inference/Bridge_Satpathy/29Aug23/3744376661_20230829-175127/balanced_genotype_microglia/0.05/raw/balanced_genotype_microglia-CASTJ.0.05.raw.h5ad
/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/hdwgcna/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/CASTJ
Done with /cellar/users/aklie/data/igvf/topic_grn_links/subsets/for_regulon_infere

In [27]:
sc.read_h5ad("/cellar/users/aklie/data/igvf/topic_grn_links/subsets/for_regulon_inference/Bridge_Satpathy/29Aug23/3744376661_20230829-175127/balanced_genotype_microglia/0.05/raw/balanced_genotype_microglia-B6J.0.05.raw.h5ad").layers["normalized_counts"][:10, :10].todense()

matrix([[3. , 0. , 0. , 0. , 0. , 2. , 1. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ],
        [1. , 0. , 0. , 0. , 2. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ],
        [1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]], dtype=float32)

In [29]:
sc.read_h5ad("/cellar/users/aklie/data/igvf/topic_grn_links/subsets/for_regulon_inference/Bridge_Satpathy/29Aug23/3744376661_20230829-175127/balanced_genotype_microglia/0.05/pf_log1p_pf/balanced_genotype_microglia-B6J.0.05.pf_log1p_pf.h5ad").layers["normalized_counts"][:10, :10]

array([[0.561957  , 0.        , 0.        , 0.        , 0.        ,
        0.4125561 , 0.23110768, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.87245625, 0.87245625, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [1.7418704 , 0.        , 0.        , 0.        , 2.4305923 ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [1.0639343 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.0639343 ],
       [0.88486373, 0.        , 0.       